# ReEDS Scenarios on PV ICE Tool

To explore different scenarios for furture installation projections of PV (or any technology), ReEDS output data can be useful in providing standard scenarios. ReEDS installation projections are used in this journal as input data to the PV ICE tool. 

Current sections include:

<ol>
    <li> ### Reading a standard ReEDS output file </li>
    <li> ### Saving PCA data as PV ICE input format </li>
    <li> ### Saving State data as PV ICE input format </li>
</ol>


In [2]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 8)

In [3]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent.parent / 'PV_ICE' / 'TEMP')

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\TEMP


## Reading a simple standard ReEDS output file

In [4]:
#We need to come up with a default location to save ReEDS output files
#reedsFile = str(Path().resolve().parent.parent.parent.parent / 'December Core Scenarios ReEDS Outputs Solar Futures v3a.xlsx')
#print ("Input file is stored in %s" % reedsFile)
reedsFile = str(Path().resolve().parent.parent.parent/ 'PV_ICE' / 'baselines' / 'SupportingMaterial' / '100REby2035-ReEDS.csv')
print ("Input file is stored in %s" % reedsFile)

Input file is stored in C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\baselines\SupportingMaterial\100REby2035-ReEDS.csv


In [5]:
REEDSInput = pd.read_csv(reedsFile)
REEDSInput.head

<bound method NDFrame.head of       scenario  year   PCA  Capacity (GW)
0     a1_RE100  2010    p1       0.000016
1     a1_RE100  2010   p10       0.486023
2     a1_RE100  2010  p100       0.000000
3     a1_RE100  2010  p101       0.021315
4     a1_RE100  2010  p102       0.014625
...        ...   ...   ...            ...
8437    a1_Ref  2050   p95      38.563516
8438    a1_Ref  2050   p96      16.298854
8439    a1_Ref  2050   p97      18.744921
8440    a1_Ref  2050   p98       6.378485
8441    a1_Ref  2050   p99      28.449829

[8442 rows x 4 columns]>

# Aggregate down to simple MW Installed input

In [61]:
rawdf = REEDSInput.copy()

#currently, we're ignoring region, so drop PCA
rawdf.drop(columns=['PCA'], inplace=True)

#aggregate and sum by scenarios and year to get an annual (bi-annual) installation by scenario
df = rawdf.groupby(['scenario', 'year'])['Capacity (GW)'].sum()
df_evens = pd.DataFrame(df)
df_evens_byscen = df.unstack(level='scenario')
#df_evens.head(25) #this df contains all the scenarios, even years only

In [92]:
#Now create the odd years of the annual/biannual data
df_odds = pd.DataFrame(df)
#create a series of the odd year values
odds = pd.Series(df_odds.index.levels[1]-1)
df_odds.index = df.index.set_levels(odds, level=1)
df_odds_byscen = df_odds.unstack(level='scenario')
print(df_odds_byscen)

         Capacity (GW)                         
scenario      a1_RE100      a1_RE95      a1_Ref
year                                           
2009          1.246592     1.246592    1.246592
2011          6.242318     6.242318    6.242318
2013         16.090783    16.090783   16.090783
2015         34.276154    34.276154   34.276154
2017         52.093001    52.093001   52.093001
2019         83.778187    83.778187   83.533329
2021        146.553694   146.553694  102.993795
2023        191.078858   191.078858  128.602753
2025        326.268392   326.268392  177.112068
2027        431.634770   431.634770  230.392738
2029        580.819921   580.819921  309.015319
2031        702.410947   702.410947  333.157319
2033        812.257622   812.257622  354.101299
2035        934.894845   934.894845  385.447237
2037       1078.186255  1078.186255  417.757467
2039       1237.253347  1237.253347  464.569087
2041       1283.271124  1283.384991  512.675339
2043       1320.858174  1322.752809  569

In [100]:
#put the evens and odds together, sort by year
df_allyrs = pd.concat([df_evens, df_odds])
df_allyrs.sort_index(axis=0, level=1, inplace=True)
df_allyrs_byscen = df_allyrs.unstack(level=0)
#print(df_allyrs_byscen)

#modify by DC:AC ratio(1.2 avg) and 85% average marketshare of c-Si technology and convert to MW
df_cSi_installs = df_allyrs_byscen*1.2*0.85*1000
#print(df_cSi_installs)

In [ ]:
#output the file
df_cSi_installs.to_csv('output_reeds4PVICE.csv', index=True)

# Append to Projections Options File

## Save Input Files by PCA

#### Create a copy of the REEDS Input and modify structure for PCA focus

#### Loading Module Baseline. Will be used later to populate all the columsn otehr than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model

In [6]:
import PV_ICE
r1 = PV_ICE.Simulation(name='Simulation1', path=testfolder)
r1.createScenario(name='US', file=r'..\baselines\SolarFutures_2021\baseline_modules_US_Reeds.csv')
baseline = r1.scenario['US'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head()

path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repair,mod_MerchantTail,mod_Reuse
year,,,,,,,,,,,
2010,14.7,25.0,30.0,0.3,22.7,98.0,0.0,0.0,0.0,0.0,0.0
2011,15.1,25.0,30.0,0.3,23.0,98.0,0.0,0.0,0.0,0.0,0.0
2012,15.4,25.0,30.0,0.3,23.5,98.0,0.0,0.0,0.0,0.0,0.0
2013,16.0,28.0,33.0,0.3,24.2,98.0,0.0,0.0,0.0,0.0,0.0
2014,16.3,28.0,33.0,0.3,26.0,98.0,0.0,0.0,0.0,0.0,0.0


#### For each Scenario and for each PCA, combine with baseline and save as input file

In [8]:
for ii in range (len(rawdf.unstack(level=1))):
    PCA = rawdf.unstack(level=1).iloc[ii].name[1]
    SCEN = rawdf.unstack(level=1).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+PCA +'.csv'
    subtestfolder = os.path.join(testfolder, 'PCAs')
    if not os.path.exists(subtestfolder):
        os.makedirs(subtestfolder)
    filetitle = os.path.join(subtestfolder, filetitle)
    A = rawdf.unstack(level=1).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 0.85
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
   
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repair,mod_MerchantTail,mod_Reuse\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)

## Save Input Files By States

#### Reassign data from REEDS Input, as we need one of the columns we dropped.

In [9]:
rawdf = REEDSInput.copy()
#rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year','PCA', 'State'], inplace=True)
rawdf.head(21)


Capacity (GW)
Scenario     Year PCA State               
95-by-35.Adv 2010 p1  WA          0.000017
             2011 p1  WA          0.005158
             2012 p1  WA          0.005158
             2013 p1  WA          0.007146
             2014 p1  WA          0.007146
             2015 p1  WA          0.018253
             2016 p1  WA          0.018253
             2017 p1  WA          0.019460
             2018 p1  WA          0.019460
             2019 p1  WA          0.036320
             2020 p1  WA          0.036320
             2021 p1  WA          0.002434
             2022 p1  WA          0.002434
             2023 p1  WA          0.010066
             2024 p1  WA          0.010066
             2025 p1  WA          0.049445
             2026 p1  WA          0.049445
             2027 p1  WA          0.051342
             2028 p1  WA          0.051342
             2029 p1  WA          0.105931
             2030 p1  WA          0.105931

#### Group data so we can work with the States instead

In [10]:
#df = rawdf.groupby(['Scenario','State', 'Year'])['Capacity (GW)'].sum(axis=0)
df = rawdf.groupby(['Scenario','State', 'Year'])['Capacity (GW)'].sum()
df = pd.DataFrame(df)
df.head()

Capacity (GW)
Scenario          State Year               
95-by-35+Elec.Adv AL    2010       0.000000
                        2011       0.000000
                        2012       0.000000
                        2013       0.000951
                        2014       0.000951

#### For each Scenario and for each STATE, combine with baseline and save as input file

In [11]:
for ii in range (len(df.unstack(level=2))):   
    STATE = df.unstack(level=2).iloc[ii].name[1]
    SCEN = df.unstack(level=2).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+STATE +'.csv'
    
    subtestfolder = os.path.join(testfolder, 'STATEs')
    if not os.path.exists(subtestfolder):
        os.makedirs(subtestfolder)
    filetitle = os.path.join(subtestfolder, filetitle)

    A = df.unstack(level=2).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 0.85 # marketshares['Si']
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    
    
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repair,mod_MerchantTail,mod_Reuse\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)

# Saving US Baseline

### Create a copy of the REEDS Input and modify structure for PCA focus

In [12]:
rawdf = REEDSInput.copy()
#rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year'], inplace=True)
rawdf.head(21)

PCA State  Capacity (GW)
Scenario     Year                         
95-by-35.Adv 2010  p1    WA       0.000017
             2011  p1    WA       0.005158
             2012  p1    WA       0.005158
             2013  p1    WA       0.007146
             2014  p1    WA       0.007146
             2015  p1    WA       0.018253
             2016  p1    WA       0.018253
             2017  p1    WA       0.019460
             2018  p1    WA       0.019460
             2019  p1    WA       0.036320
             2020  p1    WA       0.036320
             2021  p1    WA       0.002434
             2022  p1    WA       0.002434
             2023  p1    WA       0.010066
             2024  p1    WA       0.010066
             2025  p1    WA       0.049445
             2026  p1    WA       0.049445
             2027  p1    WA       0.051342
             2028  p1    WA       0.051342
             2029  p1    WA       0.105931
             2030  p1    WA       0.105931

In [13]:
#df = rawdf.groupby(['Scenario','Year'])['Capacity (GW)'].sum(axis=0)
df = rawdf.groupby(['Scenario','Year'])['Capacity (GW)'].sum()


### Loading Module Baseline. Will be used later to populate all the columsn other than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model

In [14]:
import PV_ICE
r1 = PV_ICE.Simulation(name='Simulation1', path=testfolder)
r1.createScenario(name='US', file=r'..\baselines\SolarFutures_2021\baseline_modules_US_Reeds.csv')
baseline = r1.scenario['US'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head()

path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repair,mod_MerchantTail,mod_Reuse
year,,,,,,,,,,,
2010,14.7,25.0,30.0,0.3,22.7,98.0,0.0,0.0,0.0,0.0,0.0
2011,15.1,25.0,30.0,0.3,23.0,98.0,0.0,0.0,0.0,0.0,0.0
2012,15.4,25.0,30.0,0.3,23.5,98.0,0.0,0.0,0.0,0.0,0.0
2013,16.0,28.0,33.0,0.3,24.2,98.0,0.0,0.0,0.0,0.0,0.0
2014,16.3,28.0,33.0,0.3,26.0,98.0,0.0,0.0,0.0,0.0,0.0


### For each Scenario, combine with baseline and save as input file¶

In [15]:
for ii in range (len(df.unstack(level=1))):
    SCEN = df.unstack(level=1).index[ii]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'.csv'
    
    subtestfolder = os.path.join(testfolder, 'USA')
    if not os.path.exists(subtestfolder):
        os.makedirs(subtestfolder)
    filetitle = os.path.join(subtestfolder, filetitle)
    
    A = df.unstack(level=1).iloc[ii]

    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 0.85 # marketshares['Si']
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
   
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repair,mod_MerchantTail,mod_Reuse\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)